In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [4]:
loader = PyPDFLoader("10Q.pdf")
pages = loader.load()

In [6]:
text = ''
for page in pages:
    text+=page.page_content

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [24]:
len(chunks)

14

In [9]:
from langchain.chat_models import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "API_KEY"


In [11]:
llm=ChatOpenAI(model_name='gpt-3.5-turbo')

In [13]:
llm.get_num_tokens(text)

18141

In [19]:
from llamaapi import LlamaAPI

# Replace 'Your_API_Token' with your actual API token
llama = LlamaAPI("llm_api")

In [21]:
from langchain_experimental.llms import ChatLlamaAPI
llm =ChatLlamaAPI(client=llama,)

In [32]:
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id = "amitesh11/bart-finance-pegasus",
    huggingfacehub_api_token = "hug-api"
)

C:\Users\kinga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.llms import Llama
api_key = 'llama_api'
model_version = 'llama2-70b'
llm = Llama(
    api_key=api_key,
    model_version=model_version
)

In [ ]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose = False
)

summary = chain.run(chunks)